In [ ]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 13.8 MB/s eta 0:00:00


In [ ]:
import onnx

import numpy as np
import pandas as pd
import tensorflow as tf
import onnxruntime

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from onnxruntime import InferenceSession
from onnx import numpy_helper

model_path = 'rpmodel.onnx'
onnx_model = onnx.load(model_path)
try:
	onnx.checker.check_model(onnx_model)
except onnx.checker.ValidationError as e:
	pass
else:
	print('model is valid')
	
# Download the dataset
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)
raw_data = dataframe.values
dataframe.head()

# The last element contains the labels
labels = raw_data[:, -1]

# The other data points are the electrocadriogram data
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=21)


model is valid


In [ ]:
test_data = np.float32(test_data)

In [ ]:
td = test_data[1]
td = np.reshape(td, )
print(td.shape)

(140,)


In [ ]:
#ortvalue = onnxruntime.OrtValue.ortvalue_from_numpy(test_data)
ortvalue = onnxruntime.OrtValue.ortvalue_from_numpy(td)
ortvalue.device_name()  # 'cpu'
ortvalue.shape()        # shape of the numpy array X
ortvalue.data_type()    # 'tensor(float)'
ortvalue.is_tensor()    # 'True'
np.array_equal(ortvalue.numpy(), test_data)  # 'True'

False

In [ ]:
ortvalue.data_type()    # 'tensor(float)'

'tensor(float)'

In [ ]:

session = onnxruntime.InferenceSession('rpmodel.onnx', None)

In [83]:
Input_name = session.get_inputs()[0].name
result = session.run([], {Input_name: ortvalue})

In [ ]:
normal = lambda x: x[1] > 0.7

In [88]:
#print("generated input numpy data type", test_data.dtype)
tensortest = numpy_helper.from_array(test_data)
#print(test_data)
#print(""tensortest.data_type)
# Input_type = session.get_inputs()[0].type
# print("Input type", Input_type)


output_name = session.get_outputs()[0].name
#Input_name = session.get_inputs()[0].name


result = session.run([output_name], {Input_name: test_data})
pred = result[0]
#prediction=np.argmax(np.array(result).squeeze(), axis=0)
prediction = [normal(i) for i in pred]
print(prediction)
for i in prediction:
  print('Normal' if i else 'Abnormal')
#print('normal' if prediction == True else 'abnormal')
#print(pred)

[True, True, True, True, False, True, True, True, False, True, True, True, False, True, False, True, True, True, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, True, True, False, True, False, True, True, True, True, True, True, False, True, True, False, False, False, True, False, True, True, False, True, False, True, True, False, False, True, False, False, True, True, False, True, False, False, True, False, False, True, True, True, True, False, False, True, True, True, False, False, False, True, True, False, False, False, True, False, True, False, False, True, False, True, False, True, True, True, False, True, True, True, True, True, False, True, False, True, True, False, False, True, False, True, True, True, True, True, False, True, True, True, False, True, False, True, False, True, True, True, True, True, False, False, True, False, False, True, True, True, True, True, False, True, True, False, True, True, False, 

In [ ]:
#print(result[0])

[[0.00771706 0.9922829 ]
 [0.00772728 0.99227273]
 [0.00771706 0.9922829 ]
 ...
 [0.00771706 0.9922829 ]
 [0.00771706 0.9922829 ]
 [0.00771706 0.9922829 ]]
